In [ ]:
ENDPOINT_WHITELIST = "https://collaborative-training-auth.huggingface.co"

# Packages import

In [ ]:
!pip install requests==2.24.0 cryptography==3.1.1 huggingface_hub==0.0.2 pydantic==1.8.1

# Code stuff

In [ ]:
import pprint
from getpass import getpass
from typing import List, Optional

import requests
from huggingface_hub import HfApi
from huggingface_hub.hf_api import HfApi, HfFolder
from pydantic import BaseModel, IPvAnyAddress, validator
from requests.exceptions import HTTPError

In [ ]:
class HFTokenManagemment:
    def __init__(self):
        self._api = HfApi()

    def login(self):
        username = input("Experiment owner HF username: ")
        password = getpass()
        try:
            token = self._api.login(username, password)
        except HTTPError as e:
            # probably invalid credentials, display error message.
            print(e)
            print(e.response.text)
            exit(1)
        HfFolder.save_token(token)
        print("Login successful")
        print("Your token:", token, "\n")
        print("Your token has been saved to", HfFolder.path_token)

    def logout(self):
        token = HfFolder.get_token()
        if token is None:
            print("Not logged in")
            exit()
        HfFolder.delete_token()
        self._api.logout(token)
        print("Successfully logged out.")

In [ ]:
# Define models to help request to API
class UserCreate(BaseModel):
    """
    username are required for registering a new user
    """

    username: str


class ExperimentFullUpdate(BaseModel):
    name: Optional[str]
    coordinator_ip: Optional[str]
    coordinator_port: Optional[int]
    added_collaborators: Optional[List[UserCreate]]
    removed_collaborators: Optional[List[UserCreate]]

    @validator("coordinator_port")
    def validate_port(cls, port):
        if port is None:
            return port

        if int(port) > 2 ** 16:
            raise ValueError("port overflow")
        return port

In [ ]:
def update_coordinator_endpoint(
    id_experiment: int, coordinator_ip: str, coordinator_port: int
):
    experiment_full_update = ExperimentFullUpdate(
        coordinator_ip=coordinator_ip, coordinator_port=coordinator_port
    )

    path = f"{ENDPOINT_WHITELIST}/api/experiments/{id_experiment}/"

    headers = {"Authorization": f"Bearer {HfFolder.get_token()}"}
    r = requests.put(
        path,
        headers=headers,
        json={
            "experiment_full_update": experiment_full_update.dict(exclude_unset=True)
        },
    )
    r.raise_for_status()
    return r.json()


def add_collaborators(id_experiment: int, collaborators_username_list: List[str]):
    new_collaborators_list = [
        UserCreate(username=username) for username in collaborators_username_list
    ]
    experiment_full_update = ExperimentFullUpdate(
        added_collaborators=new_collaborators_list
    )

    path = f"{ENDPOINT_WHITELIST}/api/experiments/{id_experiment}/"

    headers = {"Authorization": f"Bearer {HfFolder.get_token()}"}
    r = requests.put(
        path,
        headers=headers,
        json={
            "experiment_full_update": experiment_full_update.dict(exclude_unset=True)
        },
    )
    r.raise_for_status()
    return r.json()


def remove_collaborators(id_experiment: int, collaborators_username_list: List[str]):
    removed_collaborators_list = [
        UserCreate(username=username) for username in collaborators_username_list
    ]
    experiment_full_update = ExperimentFullUpdate(
        removed_collaborators=removed_collaborators_list
    )

    path = f"{ENDPOINT_WHITELIST}/api/experiments/{id_experiment}/"

    headers = {"Authorization": f"Bearer {HfFolder.get_token()}"}
    r = requests.put(
        path,
        headers=headers,
        json={
            "experiment_full_update": experiment_full_update.dict(exclude_unset=True)
        },
    )
    r.raise_for_status()
    return r.json()

def get_experiment(id_experiment: int):
    path = f"{ENDPOINT_WHITELIST}/api/experiments/{id_experiment}/"

    headers = {"Authorization": f"Bearer {HfFolder.get_token()}"}
    r = requests.get(
        path,
        headers=headers
    )
    r.raise_for_status()
    return r.json()

# Login to HF account

In [ ]:
# As a HF user, you ask a HF token session
token_management = HFTokenManagemment()
token_management.login()

In [ ]:
id_experiment = 15  # Change ME

# Get the content of the experiment
To see the collaborators already whitelisted

In [ ]:
response = get_experiment(
    id_experiment=id_experiment,
)
response

In [ ]:
print("List of collaborators in allowlist who already ask to join the collaborative training:")
for collaborator in response['collaborators']:
    if collaborator["peer_public_key"] is not None:
        print(f"  {collaborator['username']}")
              
print("\nList of collaborators in allowlist who didn't ask to join the collaborative training yet:")
for collaborator in response['collaborators']:
    if collaborator["peer_public_key"] is None:
        print(f"  {collaborator['username']}")

# Add new collaborators to the whitelist

In [ ]:
collaborators_username_list = [
    "Claire",  # Change Me
    "Jean",  # Change Me
]
add_collaborators(
    id_experiment=id_experiment, collaborators_username_list=collaborators_username_list
)

# Remove collaborators from the whitelist

In [ ]:
collaborators_username_list = [
    "Claire",  # Change Me
]
remove_collaborators(
    id_experiment=id_experiment, collaborators_username_list=collaborators_username_list
)

# Update coordinator endpoint values

In [ ]:
update_coordinator_endpoint(
    id_experiment=id_experiment,
    coordinator_ip=str,  # Change ME, ex: "1.1.1.1"
    coordinator_port=int,  # Change ME, ex: 50
)

# Logout 
Really import to run this last cell for security reasons

In [ ]:
# As a HF user, you terminate your session (your token is not longer valid)
token_management.logout()